<a href="https://colab.research.google.com/github/Billy-24/Billy-24/blob/main/projet_de_vin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, pathlib, PIL, shutil, glob
from google.colab import files
import seaborn as sns
import numpy as np
from pandas.core.dtypes.api import is_numeric_dtype, is_string_dtype
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, FunctionTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import boxcox
from scipy.stats import pearsonr
import graphviz
from yellowbrick.features import pca_decomposition


In [ ]:
import pandas as pd
from google.colab import files
import io
uploaded = files.upload()
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore

In [ ]:
# L'analyse exploratoire des données
data = pd.read_csv(io.BytesIO(uploaded['winemag-data-130k-v2 (3).csv']))
data.head()

In [ ]:
data.shape

In [ ]:
# type de variable du dataset
data.info()

In [ ]:
data.region_1.nunique()

In [ ]:
data.isnull().sum()

In [ ]:
# Supprimer les colonnes non pertinentes
data = data.drop(['Unnamed: 0','region_2', 'description', 'taster_twitter_handle'], axis=1)
data.head()

In [ ]:
data.describe(include="all")

In [ ]:
# vérification des valeurs manquantes dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(data.isna(), cbar=False)

In [ ]:
# Verification des doublons
duplicatas = data[data.duplicated()]
print("Enregistrements Dupliqués :", duplicatas.shape)

In [ ]:
#supprimons les doublons
data.drop_duplicates(inplace=True)

In [ ]:
data.info()

In [ ]:
# L'ensemble de données présente toutes les variables prédictives comme catégoriques, à l'exception du « prix ». La variable dépendante est « points ».cat_data=[]
# Sectionner les catégories des valeur numériques
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data)
cat_data

In [ ]:
# Transposer les valeurs manquantes sectionnées
cat_data=[]
num_data=[]
for i,c in enumerate(data.dtypes):
  if c==object:
    cat_data.append(data.iloc[:,i])
  else:
    num_data.append(data.iloc[:,i])
cat_data=pd.DataFrame(cat_data).transpose()
num_data=pd.DataFrame(num_data).transpose()
cat_data

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(num_data.isna(), cbar=False)

In [ ]:
# Analyse univariée des valeur catégoriele
cat_data = ['country', 'taster_name']

# Créer une grille de sous-graphiques en fonction du nombre de catégories
fig, axes = plt.subplots(nrows=1, ncols=len(cat_data), figsize=(12, 4))

# Parcourir la liste des catégories et créer un countplot pour chaque catégorie
for i, column in enumerate(cat_data):
    sns.countplot(x=column, data=data, ax=axes[i])
    axes[i].set_xticklabels([])
    axes[i].set_title(f'Countplot for {column}')

plt.show()


In [ ]:
# Analyse des valeurs numériques
data[['points','price']].describe()

In [ ]:
# Analyser la correlation entre price et points
data[['price','points']].corr(method = 'pearson')

In [ ]:
# Carte de corrélation
plt.figure(figsize=(2,2),)
correlacion = data.corr()
sns.heatmap(correlacion,annot=True)

In [ ]:
# Suite à ce résultat 'il existe une corrélation positive modérée entre 'price' et 'points' dans vos données. Cela suggère qu'il y a une tendance générale selon laquelle des vins avec des prix plus élevés ont tendance à obtenir des points plus élevés, et vice versa. Cependant, d'autres facteurs peuvent également influencer la notation des vins, car la corrélation n'est pas parfaite.

In [ ]:
# Analyse country - points
data[['points','country']].describe()

In [ ]:
top_countries = data.groupby(['country'])['points'].mean().sort_values(ascending=False).head(10)
print(top_countries)

In [ ]:
# Analyse des variables numériques
# Analyse univariée des valeurs numériques : Créez un nuage de points avec 'price' sur l'axe des x
plt.scatter(range(len(data['price'])), data['price'])
plt.show()

In [ ]:
# Étape 1 : Supprimer la valeur aberrante dans 'price'
seuil_aberrant = 3200
num_data = num_data[num_data['price'] <= seuil_aberrant]

In [ ]:
# Nuage de points après suppression de la valeur aberrante
plt.scatter(range(len(num_data['price'])), num_data['price'])
plt.title('Nuage de points après suppression de la valeur aberrante')
plt.xlabel('Index')
plt.ylabel('Price')
plt.show()

In [ ]:
#remplaçons les valeur manquante numérique par le mean
num_data.fillna(method = 'bfill' , inplace= True)
num_data.isnull().sum().any()

In [ ]:
# Étape 4 : Remplacer les valeurs manquantes dans le reste de 'cat_data' par le mode
cat_data.fillna(cat_data.mode().iloc[0], inplace=True)

In [ ]:
# Afficher les valeurs manquantes après le remplacement par la moyenne dans 'price'
print("Valeurs manquantes dans 'price' après remplacement par la moyenne:")
print(num_data['price'].isnull().sum())

# Afficher les valeurs manquantes dans le reste de 'num_data'
print("Valeurs manquantes dans le reste de 'num_data' après remplacement par la moyenne:")
print(num_data.isnull().sum())


In [ ]:
# Un test pour voir si on retrouve encore des NAN dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(num_data.isna(), cbar=False)

In [ ]:
# Analyse
## le « prix » présente une grande quantité de données atypiques et une large marge. Avec la transformation logarithmique, un comportement plus normalisé est obtenu et la limite des données aberrantes est réduite. Cependant, la transformation boxcox, qui est plus généralisée et a des valeurs nulles et négatives, présente une normalisation légère mais meilleure que la transformation logarithmique.
##La variable créée 'quality_price_ratio' présente un comportement qui tend à se normaliser, pour améliorer cette normalisation, une transformation sera effectuée. #

In [ ]:
#data['quality_price_ratio'] = data['points'] / data['price']#

In [ ]:
# Réinitialiser les index des DataFrames
num_data.reset_index(drop=True, inplace=True)
cat_data.reset_index(drop=True, inplace=True)

# Fusionner (concaténer) les deux DataFrames
data_final = pd.concat([num_data, cat_data], axis=1)
data_final

In [ ]:
#fusionner dataset à cette étape (combined_data)
combined_data = pd.concat([cat_data, num_data], axis=1)

combined_data

In [ ]:
num_data

In [ ]:
# Un test pour voir si on retrouve encore des NAN dans le dataset
plt.figure(figsize=(20,10))
sns.heatmap(data_final.isna(), cbar=False)

In [ ]:
combined_data = pd.DataFrame(data)

# Encoder les variables catégorielles avec l'encodage one-hot
combined_data_encoded = pd.get_dummies(combined_data, columns=['country', 'designation', 'province', 'region_1', 'taster_name', 'variety', 'winery'])

# Afficher le DataFrame encodé
print(combined_data_encoded)

In [ ]:
### Training pipeline ###
print("--- Training pipeline ---")
print()

# Encoding categorical features and standardizing numeric features

print("#### X_train BEFORE preprocessing ####")
print(combined_data.head())
print()

print("Encoding categorical features and standardizing numerical features...")
## First let's import libraries
## StandardScaler to scale data (i.e apply Z-score)
## OneHotEncoder to encode categorical variables


numeric_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History'] # Choose which column index we are going to scale
numeric_transformer = StandardScaler()

categorical_features = ['Gender', 'Married', 'Dependents', 'Education','Self_Employed','Property_Area'] # Choose which column index we are going to encode
categorical_transformer = OneHotEncoder(drop="first")

# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features), ('num', numeric_transformer, numeric_features)])
# Apply ColumnTransformer to create a pipeline that will apply the above preprocessing
feature_encoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
        ]
    )